PreProcess the data

In [ ]:
import netCDF4 as nc
import os
import ipywidgets as widgets
import numpy as np

In [ ]:

class data_converter():
    def __init__(self):
        file = ""
        path = ""
        self.default_path_input = os.getenv("HOST_VOLUME_PATH")
        self.default_path_output = os.getenv("HOST_VOLUME_PATH","converted_data")
        self.create_widgets()
        self._setup_callbacks()



    def create_widgets(self):
        self.txt_title = widgets.HTML('<H1>Choisissez vos parametres:</H1>')

        self.w_path = widgets.Text(value=self.default_path_input)
        self.path = self._bold_widget("Path", self.w_path)

        self.w_files = widgets.Text(placeholder="ex: file1.cdf, file2.cdf")
        self.files = self._bold_widget("Files", self.w_files)

        self.w_output_path = widgets.Text(value=self.default_path_output)
        self.output_path = self._bold_widget("Output path", self.w_output_path)

        # Run button and output
        self.run_button = widgets.Button(description="Run")
        self.output = widgets.Output()
    
    def _setup_callbacks(self):
        """Set up callbacks for interactive widgets."""
        self.run_button.on_click(self._on_button_click)
    
        
    def _bold_widget(self, value, widget):
        """Create a widget with bold label."""
        bold = widgets.HTML(value=f'<b>{value}</b>')
        return widgets.HBox([bold, widget])
    
    def get_files_from_folder(self, path):
        """Get all CDF files from a folder."""
        if os.path.isdir(path):
            return [f for f in os.listdir(path) if f.endswith(".cdf")]
        else:
            return []
    

    def check_path(self, path, files_list, output_path):
        """Check if the files exist and are readable."""
        if not os.path.isdir(path):
            print(f"Erreur : Le chemin '{path}' n'est pas un répertoire valide.")
            return
        if not os.access(path, os.R_OK):
            print(f"Erreur : Permission refusée pour accéder au répertoire '{path}'")
            return
        if not os.path.exists(output_path):
            try:
                os.makedirs(output_path)
                print(f"Créé : {output_path}")
            except PermissionError:
                print(f"Erreur : Permission refusée pour créer le répertoire '{output_path}'")
                return
   
        if not path:
            print("Erreur : Aucun chemin sélectionné.")
            return
        
        if files_list is None:
            files_list = self.get_files_from_folder(path)
        
        print(f"Files to convert: {files_list}")

        
        files_list = [file.strip() for file in files_list if file.strip()]
        print(f"Fichiers à analyser : {files_list}")
        return files_list
      
        
            
    def read_cdf_to_npy(self, path, files_list, output_path):
        print(f"Path: {output_path}")
        
        files_list_checked = self.check_path(path, files_list, output_path)
        if files_list_checked is None:
            return
        # data = nc.Dataset(self.file + self.path, 'r')
        for file in files_list_checked:
            full_path = os.path.join(path, file)
            if not os.path.isfile(full_path):
                print(f"Erreur : Le fichier '{file}' est introuvable dans '{path}'")
                return
            if not os.access(full_path, os.R_OK):
                print(f"Erreur: Permission refusée pour accéder à '{file}' dans '{path}'")
                return
            if not file.endswith('.cdf'):
                print(f"Erreur : Le fichier '{file}' n'est pas un fichier CDF valide.")
                return
            with nc.Dataset('mon_fichier.cdf', 'r') as dataset:
                data_npy = {
                    'scan_acquisition_time': dataset['scan_acquisition_time'][:],
                    'mass_values': dataset['mass_values'][:],
                    'intensity_values': dataset['intensity_values'][:],
                    'total_intensity': dataset['total_intensity'][:],
                    'point_count': dataset['point_count'][:],
                    'mass_range_min': dataset['mass_range_min'][:],
                    'mass_range_max': dataset['mass_range_max'][:],
                    'scan_number': dataset.dimensions['scan_number'].size,
                }

            #save the data to a .npy file
            base_name = f'{output_path}{file[:-4]}.npy'
            np.save(base_name, data_npy)
            print(f"Converted {file} to {base_name}")
            
    
    def _on_button_click(self, b):
        """Handle button click event to start analysis."""
        with self.output:
            self.output.clear_output()

            files_list = self.w_files.value.split(",")
        
            if files_list == ['']:
                files_list = None
                
            self.read_cdf_to_npy(self.w_path.value, files_list, self.w_output_path.value)


    def display(self):
        """Display the UI."""
        display(self.txt_title, self.path, self.files, self.output_path, self.run_button, self.output)
    

In [ ]:
data_converter = data_converter()
data_converter.display()